# 基于RNN的数字识别

### 载入MNIST数据

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('../../proj2/第二天/练习/MNIST_data', one_hot=True)

In [ ]:
sess = tf.InteractiveSession()

In [ ]:
x = tf.placeholder(tf.float32, shape=[None, 784])
y_ = tf.placeholder(tf.float32, shape=[None, 10])

In [ ]:
def lstm_cell(lstm_size):
  return tf.contrib.rnn.BasicLSTMCell(lstm_size)

在这一部分的循环神经网络中，我们选择使用LSTM结构，这种结构能够有效的防止梯度消失的问题。以上函数能够根据指定的大小返回一个LSTM单元

In [ ]:
stacked_lstm = tf.contrib.rnn.MultiRNNCell(
    [lstm_cell(28*28) for _ in range(3)]) # 参数是： cells, state_is_tuple=True

### 通过使用tf.contrib.rnn.MultiRNNCell，我们可以将多个LSTM连接起来形成多层RNN网络。


In [ ]:
state = stacked_lstm.zero_state(batch_size=tf.shape(x)[0], dtype=tf.float32)

定义RNN的初始状态为零，通过tf.shape函数得到输入的batch大小。
zero_state(
    batch_size,
    dtype
)

In [ ]:
output = x
for i in range(5):
    output, state = stacked_lstm(output, state)
print(output.shape)

将RNN在时间上展开，这里总共展开了5步，output为网络输出。

In [ ]:
y_rnn = tf.layers.dense(output, 10,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=0.1),
                    bias_initializer=tf.constant_initializer(0.1),
                    name='rnn_output_layer'
                    )

将RNN的输出经过全连接层，转化为对应的维度（10维，对应数字0-9）

In [ ]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y_rnn))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_rnn, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(2000):
        batch = mnist.train.next_batch(50)
        print(batch[0].shape)
        _,acc=sess.run([train_step,accuracy],feed_dict={x: batch[0], y_: batch[1]})
        if i % 10 == 0:            
            print('step %d, training accuracy %g' % (i, acc))
    ac=sess.run(accuracy,feed_dict={x: mnist.test.images, y_: mnist.test.labels})
    print('test accuracy %g' %ac)